In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler 
from sklearn.decomposition import PCA

In [3]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='NQbemMNI-QTC9xGVYu9x-E2eRRX8tMdmiVWLlilkYGpf',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'rainfallpredmodel-donotdelete-pr-vi05jee9b7addx'
object_key = 'weatherAUS.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [4]:
df[ "MinTemp"].fillna(df["MinTemp"].mean(),inplace=True)
df[ "MaxTemp"].fillna(df["MaxTemp"].mean(),inplace=True)
df[ "Rainfall"].fillna(df["Rainfall"].mean(),inplace=True)
df[ "Evaporation"].fillna(df["Evaporation"].mean(),inplace=True)
df[ "Sunshine"].fillna(df["Sunshine"].mean(),inplace=True)
df[ "Pressure9am"].fillna(df["Pressure9am"].mean(),inplace=True)
df[ "Cloud9am"].fillna(df["Cloud9am"].mean(),inplace=True)
df[ "Cloud3pm"].fillna(df["Cloud3pm"].mean(),inplace=True)
df[ "Temp9am"].fillna(df["Temp9am"].mean(),inplace=True)
df[ "Temp3pm"].fillna(df["Temp3pm"].mean(),inplace=True)
df[ "WindGustSpeed"].fillna(df["WindGustSpeed"].mean(),inplace=True)
df[ "WindSpeed9am"].fillna(df["WindSpeed9am"].mean(),inplace=True)
df[ "WindSpeed3pm"].fillna(df["WindSpeed3pm"].mean(),inplace=True)
df[ "Humidity9am"].fillna(df["Humidity9am"].mean(),inplace=True)
df[ "Humidity3pm"].fillna(df["Humidity3pm"].mean(),inplace=True)

In [5]:
print("Unique values in WindGustDir:",df.WindGustDir.unique())
print("Unique values in WindDir9am:",df.WindDir9am.unique())
print("Unique values in WindDir3pm:",df.WindDir3pm.unique())
print("Unique values in RainToday:",df.RainToday.unique())
print("Unique values in RainTomorrow:",df.RainTomorrow.unique())

Unique values in WindGustDir: ['W' 'WNW' 'WSW' 'NE' 'NNW' 'N' 'NNE' 'SW' nan 'ENE' 'SSE' 'S' 'NW' 'SE'
 'ESE' 'E' 'SSW']
Unique values in WindDir9am: ['W' 'NNW' 'SE' 'ENE' 'SW' 'SSE' 'S' 'NE' nan 'SSW' 'N' 'WSW' 'ESE' 'E'
 'NW' 'WNW' 'NNE']
Unique values in WindDir3pm: ['WNW' 'WSW' 'E' 'NW' 'W' 'SSE' 'ESE' 'ENE' 'NNW' 'SSW' 'SW' 'SE' 'N' 'S'
 'NNE' nan 'NE']
Unique values in RainToday: ['No' 'Yes' nan]
Unique values in RainTomorrow: ['No' 'Yes' nan]


In [6]:
df[ "WindGustDir"].fillna(df["WindGustDir"].mode()[0],inplace=True)
df[ "WindDir9am"].fillna(df["WindDir9am"].mode()[0],inplace=True)
df[ "WindDir3pm"].fillna(df["WindDir3pm"].mode()[0],inplace=True)
df[ "Pressure3pm"].fillna(df["Pressure3pm"].mode()[0],inplace=True)
df[ "RainToday"].fillna(df["RainToday"].mode()[0],inplace=True)
df[ "RainTomorrow"].fillna(df["RainTomorrow"].mode()[0],inplace=True)

In [7]:
X = df.iloc[:, :-2].values
print(X)

[['2008-12-01' 'Albury' 13.4 ... 4.509930082924903 16.9 21.8]
 ['2008-12-02' 'Albury' 7.4 ... 4.509930082924903 17.2 24.3]
 ['2008-12-03' 'Albury' 12.9 ... 2.0 21.0 23.2]
 ...
 ['2017-06-23' 'Uluru' 5.4 ... 4.509930082924903 12.5 26.1]
 ['2017-06-24' 'Uluru' 7.8 ... 2.0 15.1 26.0]
 ['2017-06-25' 'Uluru' 14.9 ... 8.0 15.0 20.9]]


In [8]:
Y = df.iloc[:, -2].values
print(Y)

['No' 'No' 'No' ... 'No' 'No' 'No']


In [9]:
from sklearn import model_selection

In [10]:
y = df['RainTomorrow']
X = df.drop(['Date','Location','WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow'], axis=1)
x_train, x_test, y_train, y_test = model_selection.train_test_split(X,y,test_size= 0.1, random_state = 0)

In [11]:
from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier
#kn=KNeighborsClassifier(n_neighbors=3)
#kn.fit(x_train,y_train)

Randm_Forest = RandomForestClassifier()
Randm_Forest.fit(x_train,y_train)
p2 = Randm_Forest.predict(x_train)

In [12]:
#p4 = kn.predict(x_train)

In [13]:
from sklearn import metrics

print("Random_Forest:", metrics.accuracy_score(y_train,p2))
#print("K-Nearest neighbors:", metrics.accuracy_score(y_train,p4))

Random_Forest: 0.9999388911804696


In [14]:
!pip install ibm_watson_machine_learning

In [35]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey": "IYhl2VJPcsKa1DvohrhA8BwB8Wk43ShPuQ1VzL0fPMXJ"
                }
client = APIClient(wml_credentials)

In [36]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [37]:
space_uid = guid_from_space_name(client, 'Project')
print("Space UID = "+space_uid)

Space UID = f03f09b8-5187-418a-a8b7-b30e0c812034


In [38]:
client.set.default_space(space_uid)

'SUCCESS'

In [49]:
client.software_specifications.list(limit=150)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [46]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [51]:
model_details = client.repository.store_model(model=Randm_Forest, meta_props = {
    client.repository.ModelMetaNames.NAME:"rainfall_pred",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid,
    client.repository.ModelMetaNames.TYPE:"scikit-learn_1.0"}
                                             )
model_id = client.repository.get_model_id(model_details)

This method is deprecated, please use get_model_id()


In [52]:
model_id

'98a98dd5-9a52-445e-8c52-2813a927085c'